In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import datetime
from sklearn.preprocessing import LabelEncoder

# Load the data
df = pd.read_csv('supermarket_data.csv')

# Pivot the data to create a user-item matrix
matrix = df.pivot_table(index='User ID', columns='Product Name', values='Interaction Type', fill_value=0)

# Calculate the user-user similarity matrix
user_similarity = pd.DataFrame(cosine_similarity(matrix), index=matrix.index, columns=matrix.index)

def collaborative_filtering(user_id, n_recommendations=3):
    """
    Collaborative filtering recommendation algorithm
    
    Parameters:
    user_id (int): The ID of the target user
    n_recommendations (int): The number of recommendations to return (default is 3)
    
    Returns:
    A list of tuples containing the recommended products and their predicted interaction score
    """
    # Get the similarity scores of the target user with other users
    sim_scores = user_similarity[user_id].sort_values(ascending=False)
    
    # Get the top n most similar users
    top_users = sim_scores.iloc[1:n_recommendations+1].index
    
    # Get the products that the target user has not interacted with
    uninteracted_products = matrix.loc[user_id][matrix.loc[user_id] == 0].index
    
    # Create a list to store the predicted interaction scores for each product
    interaction_scores = []
    
    # Calculate the predicted interaction score for each uninteracted product
    for product in uninteracted_products:
        interaction_score = np.average(matrix.loc[top_users][product], weights=sim_scores[top_users])
        interaction_scores.append((product, interaction_score))
    
    # Sort the products by their predicted interaction score in descending order
    sorted_scores = sorted(interaction_scores, key=lambda x: x[1], reverse=True)
    
    # Return the top n recommended products
    return sorted_scores[:n_recommendations]

# Get recommendations
recommended_products = collaborative_filtering(user_id=97, n_recommendations=3)
print(recommended_products)


[('Deflect-o SuperTray Unbreakable Stackable Tray, Letter, Black', 3.0), ('Adams Phone Message Book, Professional, 400 Message Capacity, 5 3/6” x 11”', 2.82988546564033), ('Logitech\xa0Gaming G510s - Keyboard', 2.668824160067906)]
